## Transit Bunching 
* `cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest`
* [Issue](https://github.com/cal-itp/data-analyses/issues/1099)
### 11/8
* Figure out how to address City of Visalia: one of the buses that is scheduled to arrive earlier arrives later than another bus. 
* This leads to a negative time stamp and makes it appear like there is a lot of bunching per the Transit Matters approach.



In [1]:
import datetime as dt

import altair as alt
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

# https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
may_date = "2024-05-22"

In [4]:
drop_for_preview = [
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "trip_id",
]

### Grab Sample Routes

In [5]:
subset = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
    "service_date",
    "frequency",
]

In [6]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

'schedule_route_dir/schedule_route_direction_metrics'

In [7]:
route_dir_columns = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "time_period",
    "route_primary_direction",
    "frequency",
    "service_date",
]

In [8]:
route_dir = merge_data.concatenate_schedule_by_route_direction([may_date])[
    route_dir_columns
]

In [9]:
route_dir.head()

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,frequency,service_date
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,Northbound,0.92,2024-05-22
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,offpeak,Northbound,0.62,2024-05-22
2,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,peak,Northbound,1.50,2024-05-22
3,015d67d5b75b5cf2b710bbadadfb75f5,17,1.00,all_day,Southbound,0.92,2024-05-22
4,015d67d5b75b5cf2b710bbadadfb75f5,17,1.00,offpeak,Southbound,0.69,2024-05-22


#### Attach operators and districts

In [10]:
# Grab Crosswalk
CROSSWALK = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

In [11]:
crosswalk_cols = [
    "schedule_gtfs_dataset_key",
    "organization_name",
    "name",
    "caltrans_district",
]

In [12]:
crosswalk_df = (
    time_series_utils.concatenate_datasets_across_dates(
        SCHED_GCS, CROSSWALK, [may_date], data_type="df", columns=crosswalk_cols
    )
    .sort_values(["service_date"])
    .reset_index(drop=True)
)

In [13]:
crosswalk_df.head(2)

,schedule_gtfs_dataset_key,organization_name,name,caltrans_district,service_date
0,1770249a5a2e770ca90628434d4934b1,Ventura County Transportation Commission,VCTC GMV Schedule,07 - Los Angeles,2024-05-22
1,f8102a9c0693206bf36d302540bf1bcf,City of Corona,Corona Schedule,08 - San Bernardino,2024-05-22


In [14]:
crosswalk_df.shape

(189, 5)

In [15]:
routes_dir2 = pd.merge(
    route_dir,
    crosswalk_df,
    on=["schedule_gtfs_dataset_key", "service_date"],
    how="left",
)

In [16]:
thousand_oaks = routes_dir2.loc[
    (routes_dir2.organization_name == "City of Thousand Oaks")
    & (routes_dir2.route_id == "3402")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [17]:
visalia = routes_dir2.loc[
    (routes_dir2.organization_name == "City of Visalia")
    & (routes_dir2.route_id == "2042")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [18]:
metro = routes_dir2.loc[
    (
        routes_dir2.organization_name
        == "Los Angeles County Metropolitan Transportation Authority"
    )
    & (routes_dir2.route_id == "204-13172")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [19]:
# '33-13172'
metro_33 = routes_dir2.loc[
    (
        routes_dir2.organization_name
        == "Los Angeles County Metropolitan Transportation Authority"
    )
    & (routes_dir2.route_id == "33-13172")
    & (routes_dir2.time_period == "all_day")
].reset_index(drop=True)

In [20]:
routes = pd.concat([thousand_oaks, visalia, metro, metro_33])

In [21]:
routes.route_id.unique()

array(['3402', '2042', '204-13172', '33-13172'], dtype=object)

In [22]:
routes.head(3)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,frequency,service_date,organization_name,name,caltrans_district
0,1770249a5a2e770ca90628434d4934b1,3402,0.00,all_day,Eastbound,1.17,2024-05-22,City of Thousand Oaks,VCTC GMV Schedule,07 - Los Angeles
1,1770249a5a2e770ca90628434d4934b1,3402,1.00,all_day,Westbound,1.17,2024-05-22,City of Thousand Oaks,VCTC GMV Schedule,07 - Los Angeles
0,3bda4652977200408690059ef2ec4b4d,2042,0.00,all_day,Eastbound,0.79,2024-05-22,City of Visalia,Visalia Schedule,06 - Fresno


### Add Trips

In [23]:
TABLE = GTFS_DATA_DICT.schedule_downloads.trips

In [24]:
FILE = f"{COMPILED_CACHED_VIEWS}{TABLE}_{may_date}.parquet"

In [25]:
trips_subset = [
    "gtfs_dataset_key",
    "route_id",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "route_long_name",
    "direction_id",
    "route_type",
]

In [26]:
trips = pd.read_parquet(FILE)[trips_subset].rename(
    columns={"gtfs_dataset_key": "schedule_gtfs_dataset_key"}
)

In [27]:
trips_routes = pd.merge(
    trips,
    routes,
    on=["schedule_gtfs_dataset_key", "route_id", "direction_id"],
    how="inner",
)

In [28]:
trips_routes.shape

(570, 15)

In [29]:
trips_routes.route_id.nunique()

4

#### I know we can get this from the warehouse but it seems cumbersome. Correct me if I'm wrong.

In [30]:
# https://gtfs.org/documentation/schedule/reference/#
route_type_crosswalk = {
    "route_type": ["0", "1", "2", "3", "4", "5", "6", "7", "11", "12"],
    "route_type_str": [
        "Tram, Streetcar, Light rail",
        "Subway, Metro",
        "Rail",
        "Bus",
        "Ferry.",
        "Cable tram.",
        "Aerial lift, suspended cable car (e.g., gondola lift, aerial tramway).",
        "Funicular.",
        "Trolleybus.",
        "Monorail.",
    ],
}

In [31]:
route_type_crosswalk_df = pd.DataFrame(route_type_crosswalk)

In [32]:
# Merge for route_type
trips_routes = pd.merge(
    trips_routes, route_type_crosswalk_df, on=["route_type"], how="left"
)

In [33]:
trips_routes = trips_routes.drop(columns=["route_type"])

In [34]:
trips_routes = trips_routes.rename(columns={"route_type_str": "route_type"})

### Get Stop Times 

In [35]:
rt_stop_times = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_rt_stop_times_2024-05-22.parquet"
)

In [36]:
rt_stop_times.head(2)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec
0,1d105244-776c-4b3f-af78-9c7ad78c2103,0b2443b6-b50f-452b-a749-464588ca93b8,8,60991.00,1fd2f07342d966919b15d5d37fda8cc8,45ae17540ca9fb5030c84dbb12e48e9a,61434
1,1d105244-776c-4b3f-af78-9c7ad78c2103,cd5650b0-9a18-4e78-aedc-385f3094fa0f,9,61179.00,1fd2f07342d966919b15d5d37fda8cc8,45ae17540ca9fb5030c84dbb12e48e9a,61616


In [37]:
trips_routes_times = pd.merge(
    rt_stop_times,
    trips_routes,
    on=[
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
    ],
    how="inner",
)

In [38]:
(trips_routes_times.scheduled_arrival_sec.isna().sum())

0

In [39]:
trips_routes_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34317 entries, 0 to 34316
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   trip_id                    34317 non-null  object        
 1   stop_id                    34317 non-null  object        
 2   stop_sequence              34317 non-null  int64         
 3   scheduled_arrival_sec      34317 non-null  float64       
 4   schedule_gtfs_dataset_key  34317 non-null  object        
 5   trip_instance_key          34317 non-null  object        
 6   rt_arrival_sec             34317 non-null  int64         
 7   route_id                   34317 non-null  object        
 8   shape_array_key            34317 non-null  object        
 9   feed_key                   34317 non-null  object        
 10  route_long_name            34317 non-null  object        
 11  direction_id               34317 non-null  float64       
 12  time

### Sorting 

In [40]:
trips_routes_times2 = trips_routes_times.sort_values(
    by=[
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "shape_array_key",
        "direction_id",
        "stop_sequence",
        "stop_id",
        "rt_arrival_sec",
    ]
).reset_index(drop=True)

In [41]:
preview_sort_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "stop_sequence",
    "rt_arrival_sec",
    "stop_id",
    "scheduled_arrival_sec",
]

In [42]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.direction_id == 1)
][preview_sort_col].shape

(7003, 7)

### Convert scheduled and RT arrival times.
* If 82800  < `scheduled_arrival_time` < 86_400 but `rt_arrival_sec` is lower say 14_000 (4 am in the morning): then the bus was scheduled to arrive on May 21 (day before the service date) but it arrived a little later on the actual service date we query the data for. 
* If  86_400 < `scheduled_arrival_time` and `rt_arrival_sec` is around 86_000 then this is the same service date. 

In [43]:
trips_routes_times2["rt_arrival_sec"].describe()

count   34317.00
mean    46219.31
std     19698.95
min        24.00
25%     31428.00
50%     46452.00
75%     61794.00
max     86399.00
Name: rt_arrival_sec, dtype: float64

In [44]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] == 86_400].shape

(11, 20)

####  Filter out for trips with `scheduled_arrival_sec` that's over 24 hours.

In [45]:
more_than_86400 = trips_routes_times2.loc[
    trips_routes_times2["scheduled_arrival_sec"] > 86_400
]

In [46]:
more_than_86400["scheduled_arrival_sec"].describe()

count     2326.00
mean     94761.44
std       5297.65
min      86460.00
25%      90180.00
50%      94320.00
75%      99000.00
max     106680.00
Name: scheduled_arrival_sec, dtype: float64

In [47]:
more_than_86400[["service_date", "rt_arrival_sec", "scheduled_arrival_sec"]]

,service_date,rt_arrival_sec,scheduled_arrival_sec
0,2024-05-22,1333,87720.00
1,2024-05-22,3229,89520.00
2,2024-05-22,4855,91320.00
3,2024-05-22,6912,93120.00
4,2024-05-22,8508,94920.00
5,2024-05-22,10300,96720.00
6,2024-05-22,12219,98520.00
7,2024-05-22,13929,100380.00
8,2024-05-22,15893,102180.00
107,2024-05-22,1436,87780.00


In [48]:
timestamp_subset = [
    "converted_schd_arrival",
    "converted_rt_arrival",
    "scheduled_arrival_sec",
    "rt_arrival_sec",
    "service_date",
]

In [49]:
def adjust_sched_arrival_seconds(sched_arrival_seconds, date, rt_arrival_sec):
    """
    Adjusts days and time of sched_arrival_seconds because it runs over 24 hours
    based on a combination of sched_arrival_seconds and rt_arrival_sec criteria.

    Parameters:
    sched_arrival_seconds (int): Number of seconds.
    date (datetime): Initial date.
    rt_arrival_sec (int): Arrival time in seconds.

    Returns:
    datetime: Adjusted date and time for sched_arrival_seconds
    """
    # If the rt arrival second is between 12-1AM and the scheduled arrival time is between
    # is between 11pm and 1am subtract a day
    if rt_arrival_sec < (60 * 60) and (82_800 < sched_arrival_seconds < 90_000):
        return pd.Timestamp(date + pd.Timedelta(days=-1)) + pd.Timedelta(
            seconds=sched_arrival_seconds % 86400
        )
    else:
        # No change
        return pd.Timestamp(date) + pd.Timedelta(seconds=sched_arrival_seconds)

#### Subset to make the df smaller


In [50]:
subset = [
    "stop_id",
    "stop_sequence",
    "scheduled_arrival_sec",
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "rt_arrival_sec",
    "route_id",
    "shape_array_key",
    "route_long_name",
    "direction_id",
    "organization_name",
    "caltrans_district",
    "service_date",
    "route_type",
    "feed_key",
]

In [51]:
trips_routes_times2 = trips_routes_times2[subset]

In [52]:
trips_routes_times2["converted_schd_arrival2"] = trips_routes_times2.apply(
    lambda row: adjust_sched_arrival_seconds(
        row["scheduled_arrival_sec"], row["service_date"], row["rt_arrival_sec"]
    ),
    axis=1,
)

In [53]:
trips_routes_times2["converted_rt_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["rt_arrival_sec"] % 86400, unit="s")

In [54]:
trips_routes_times2["converted_schd_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["scheduled_arrival_sec"] % 86400, unit="s")

#### Checkout results from the code
* Make sure it makes sense.


In [55]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] > 86_400][
    [
        "scheduled_arrival_sec",
        "converted_schd_arrival",
        "converted_schd_arrival2",
        "rt_arrival_sec",
        "converted_rt_arrival",
    ]
]

,scheduled_arrival_sec,converted_schd_arrival,converted_schd_arrival2,rt_arrival_sec,converted_rt_arrival
0,87720.00,2024-05-22 00:22:00,2024-05-21 00:22:00,1333,2024-05-22 00:22:13
1,89520.00,2024-05-22 00:52:00,2024-05-21 00:52:00,3229,2024-05-22 00:53:49
2,91320.00,2024-05-22 01:22:00,2024-05-23 01:22:00,4855,2024-05-22 01:20:55
3,93120.00,2024-05-22 01:52:00,2024-05-23 01:52:00,6912,2024-05-22 01:55:12
4,94920.00,2024-05-22 02:22:00,2024-05-23 02:22:00,8508,2024-05-22 02:21:48
5,96720.00,2024-05-22 02:52:00,2024-05-23 02:52:00,10300,2024-05-22 02:51:40
6,98520.00,2024-05-22 03:22:00,2024-05-23 03:22:00,12219,2024-05-22 03:23:39
7,100380.00,2024-05-22 03:53:00,2024-05-23 03:53:00,13929,2024-05-22 03:52:09
8,102180.00,2024-05-22 04:23:00,2024-05-23 04:23:00,15893,2024-05-22 04:24:53
107,87780.00,2024-05-22 00:23:00,2024-05-21 00:23:00,1436,2024-05-22 00:23:56


### Deal with delays
* Some very extreme values.

In [56]:
def deal_with_23_hours(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes a pandas DataFrame `df` as input, modifies it, and returns the modified DataFrame.
    The function performs two operations:
    1. It adds one day to the `converted_schd_arrival` column values if
    the corresponding `delay_min` value exceeds 23 hours (1380 minutes).
    This is done using the `apply` method with a lambda function.
    2. It recalculates the `delay_min` column by subtracting the `converted_schd_arrival`.

    Parameters:
    df (pandas.DataFrame): Input DataFrame containing the columns `converted_schd_arrival` and `delay_min`.

    Returns:
    pandas.DataFrame: Modified DataFrame with updated `converted_schd_arrival` and `delay_min` columns.

    Note: This function assumes that the input DataFrame has the required columns and that the `rt_stop_times3` DataFrame is defined in the global scope.
    """
    df["delay_min"] = (
        df["converted_rt_arrival"] - df["converted_schd_arrival"]
    ).dt.total_seconds() / 60

    display(df["delay_min"].describe())

    # If the delay is  23 hours btwn sched and rt arrival
    # subtract a day to the converted scheduled arrival
    df["converted_schd_arrival"] = df.apply(
        lambda row: row["converted_schd_arrival"] - pd.Timedelta(days=1)
        if row["delay_min"] < (23 * 60)
        else row["converted_schd_arrival"],
        axis=1,
    )

    # If the delay is more -23 hours
    # add a day to the converted scheduled arrival
    df["converted_schd_arrival"] = df.apply(
        lambda row: row["converted_schd_arrival"] + pd.Timedelta(days=1)
        if row["delay_min"] > -(23 * 60)
        else row["converted_schd_arrival"],
        axis=1,
    )

    df["delay_min"] = (
        df["converted_rt_arrival"] - df["converted_schd_arrival"]
    ).dt.total_seconds() / 60

    display(df["delay_min"].describe())
    return df

In [57]:
more_than_86400 = deal_with_23_hours(trips_routes_times2)

count   34317.00
mean        1.63
std        43.02
min     -1437.80
25%        -0.52
50%         1.10
75%         3.87
max      1439.72
Name: delay_min, dtype: float64

count   34317.00
mean        2.47
std         7.04
min       -22.75
25%        -0.52
50%         1.10
75%         3.87
max        99.27
Name: delay_min, dtype: float64

In [58]:
percentiles = [0.01, 0.02, 0.05, 0.1, 0.9, 0.95, 0.98, 0.99]

### Help, can't fix everythin...how to address these edge cases? 

In [59]:
trips_routes_times2.converted_schd_arrival.describe()

/tmp/ipykernel_2867/3770378478.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  trips_routes_times2.converted_schd_arrival.describe()


count                   34317
unique                   1452
top       2024-05-22 16:36:00
freq                       46
first     2024-05-21 23:51:00
last      2024-05-23 00:02:00
Name: converted_schd_arrival, dtype: object

In [60]:
trips_routes_times2[
    trips_routes_times2["converted_schd_arrival"].dt.strftime("%Y-%m-%d")
    == "2024-05-21"
].head(2)

,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,route_long_name,direction_id,organization_name,caltrans_district,service_date,route_type,feed_key,converted_schd_arrival2,converted_rt_arrival,converted_schd_arrival,delay_min
15852,5698,23,86100.00,0666caf3ec1ecc96b74f4477ee4bc939,de93caca29c802bfc86bc78e05953d6f,44,204-13172,6a10ede3fa469c8b4d9bf761946ed20a,Metro Local Line,0.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-21 23:55:00,2024-05-22 00:00:44,2024-05-21 23:55:00,5.73
15958,5666,24,86160.00,0666caf3ec1ecc96b74f4477ee4bc939,de93caca29c802bfc86bc78e05953d6f,77,204-13172,6a10ede3fa469c8b4d9bf761946ed20a,Metro Local Line,0.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-21 23:56:00,2024-05-22 00:01:17,2024-05-21 23:56:00,5.28


In [61]:
trips_routes_times2[
    trips_routes_times2["converted_schd_arrival"].dt.strftime("%Y-%m-%d")
    == "2024-05-23"
].head(2)

,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,route_long_name,direction_id,organization_name,caltrans_district,service_date,route_type,feed_key,converted_schd_arrival2,converted_rt_arrival,converted_schd_arrival,delay_min
1055,140960,11,86400.00,0666caf3ec1ecc96b74f4477ee4bc939,252dc54fdd5952df75e1e0902ab298f4,86383,204-13172,013c88496835cd19be42988f8925c274,Metro Local Line,1.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-23,2024-05-22 23:59:43,2024-05-23,-0.28
3835,14044,37,86400.00,0666caf3ec1ecc96b74f4477ee4bc939,242c9f205c9322a1d0639b5cf92dd18c,86313,204-13172,013c88496835cd19be42988f8925c274,Metro Local Line,1.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-23,2024-05-22 23:58:33,2024-05-23,-1.45


In [62]:
trips_routes_times2.converted_rt_arrival.describe()

/tmp/ipykernel_2867/452450045.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  trips_routes_times2.converted_rt_arrival.describe()


count                   34317
unique                  27103
top       2024-05-22 14:10:53
freq                        7
first     2024-05-22 00:00:24
last      2024-05-22 23:59:59
Name: converted_rt_arrival, dtype: object

#### Question: Last time, I received the suggestion to throw away things more than 2 hrs because that is not bunching. But wouldn't this be misleading as a metric when we want to calculate all the rows?

In [63]:
print(trips_routes_times2.delay_min.describe(percentiles))

count   34317.00
mean        2.47
std         7.04
min       -22.75
1%         -4.83
2%         -3.67
5%         -2.45
10%        -1.68
50%         1.10
90%         7.72
95%        10.60
98%        14.95
99%        18.03
max        99.27
Name: delay_min, dtype: float64


In [64]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min < 120
# ].reset_index(drop=True)

In [65]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min > -120
# ].reset_index(drop=True)

### Calculate the actual & scheduled headway the `operator-route-direction_id-stop_sequence-stop_id-` grain


In [66]:
groupby_cols = [
    "schedule_gtfs_dataset_key",
    "route_long_name",
    "direction_id",
    "stop_id",
    "stop_sequence",
]

In [67]:
trips_routes_times2["actual_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_rt_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [68]:
trips_routes_times2["scheduled_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_schd_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [69]:
trips_routes_times2["scheduled_arrival_lag_min"].describe(percentiles)

count   33641.00
mean       15.35
std       130.88
min     -1415.00
1%        -77.00
2%        -10.00
5%          9.00
10%        10.00
50%        15.00
90%        31.00
95%        50.00
98%        70.00
99%       135.00
max      1208.00
Name: scheduled_arrival_lag_min, dtype: float64

In [70]:
trips_routes_times2["actual_arrival_lag_min"].describe(percentiles)

count   33641.00
mean       15.51
std       130.71
min     -1415.03
1%        -80.54
2%          1.03
5%          4.08
10%         7.33
50%        15.02
90%        35.98
95%        50.65
98%        70.79
99%       120.52
max      1199.62
Name: actual_arrival_lag_min, dtype: float64

In [71]:
preview_time_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "stop_id",
    "stop_sequence",
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
]

In [72]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_id == "3961")
][preview_time_col]

,schedule_gtfs_dataset_key,route_id,stop_id,stop_sequence,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min
20279,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,3961,66,2024-05-22 06:24:33,NaN,2024-05-22 06:26:00,NaN
20280,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,3961,66,2024-05-22 17:00:33,636.00,2024-05-22 17:01:00,635.00


### Many lags are actually empty b/c it's the first of that groupby-sequence.

In [73]:
trips_routes_times2[trips_routes_times2["scheduled_arrival_lag_min"].isna()][
    preview_time_col
].sample()

,schedule_gtfs_dataset_key,route_id,stop_id,stop_sequence,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min
10317,0666caf3ec1ecc96b74f4477ee4bc939,33-13172,4690,21,2024-05-22 00:24:04,NaN,2024-05-22 00:25:00,NaN


In [74]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 2)
][preview_time_col]

,schedule_gtfs_dataset_key,route_id,stop_id,stop_sequence,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min
0,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 00:22:13,NaN,2024-05-22 00:22:00,NaN
1,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 00:53:49,31.60,2024-05-22 00:52:00,30.00
2,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 01:20:55,27.10,2024-05-22 01:22:00,30.00
3,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 01:55:12,34.28,2024-05-22 01:52:00,30.00
4,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 02:21:48,26.60,2024-05-22 02:22:00,30.00
5,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 02:51:40,29.87,2024-05-22 02:52:00,30.00
6,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 03:23:39,31.98,2024-05-22 03:22:00,30.00
7,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 03:52:09,28.50,2024-05-22 03:53:00,31.00
8,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 04:24:53,32.73,2024-05-22 04:23:00,30.00
9,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,140959,2,2024-05-22 04:56:32,31.65,2024-05-22 04:52:00,29.00


In [75]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 59)
][preview_time_col]

,schedule_gtfs_dataset_key,route_id,stop_id,stop_sequence,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min
6074,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 00:18:28,NaN,2024-05-22 00:19:00,NaN
6075,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 00:44:58,26.50,2024-05-22 00:46:00,27.00
6076,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 01:17:25,32.45,2024-05-22 01:15:00,29.00
6077,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 01:43:55,26.50,2024-05-22 01:43:00,28.00
6078,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 02:12:08,28.22,2024-05-22 02:13:00,30.00
6079,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 02:47:39,35.52,2024-05-22 02:43:00,30.00
6080,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 03:13:44,26.08,2024-05-22 03:13:00,30.00
6081,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 03:43:54,30.17,2024-05-22 03:43:00,30.00
6082,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 04:13:09,29.25,2024-05-22 04:13:00,30.00
6083,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,14022,59,2024-05-22 04:42:44,29.58,2024-05-22 04:43:00,30.00


In [76]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 46)
    & (trips_routes_times2.stop_id == "5685")
][preview_time_col]

,schedule_gtfs_dataset_key,route_id,stop_id,stop_sequence,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min
18284,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 00:22:02,NaN,2024-05-22 00:16:00,NaN
18285,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 00:46:39,24.62,2024-05-22 00:46:00,30.00
18286,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 01:24:30,37.85,2024-05-22 01:16:00,30.00
18287,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 01:48:25,23.92,2024-05-22 01:46:00,30.00
18288,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 02:16:15,27.83,2024-05-22 02:16:00,30.00
18289,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 02:44:30,28.25,2024-05-22 02:46:00,30.00
18290,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 03:13:04,28.57,2024-05-22 03:16:00,30.00
18291,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 03:48:36,35.53,2024-05-22 03:46:00,30.00
18292,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 04:16:06,27.50,2024-05-22 04:16:00,30.00
18293,0666caf3ec1ecc96b74f4477ee4bc939,204-13172,5685,46,2024-05-22 04:52:55,36.82,2024-05-22 04:46:00,30.00


In [77]:
# rt_stop_times4 = rt_stop_times4.fillna(0)

### Transit Matters Method

In [78]:
transit_matters_df1 = trips_routes_times2.copy()

In [79]:
transit_matters_df1["pct_actual_schd_headway"] = (
    transit_matters_df1.actual_arrival_lag_min
    / transit_matters_df1.scheduled_arrival_lag_min
)

In [80]:
transit_matters_df1["bunched_y_n"] = np.where(
    transit_matters_df1["pct_actual_schd_headway"] < 0.25, "bunched", "not bunched"
)

In [81]:
transit_matters_df1.bunched_y_n.value_counts() / len(transit_matters_df1)

not bunched   0.97
bunched       0.03
Name: bunched_y_n, dtype: float64

#### Aggregate.
* At this point, it doesn't matter the sequence, we just care about how bunched the traffic is around one partiuclar stop. 
* See how many trips for that grain are considered "bunched" or not.

In [82]:
def bunched_not_bunched(
    df: pd.DataFrame, bunched_y_n: str, groupby_cols: list
) -> pd.DataFrame:
    df2 = df.loc[df.bunched_y_n == bunched_y_n].reset_index(drop=True)

    bunched_y_n = bunched_y_n.replace(" ", "_")
    agg1 = (
        df2.groupby(groupby_cols).agg({"trip_instance_key": "nunique"}).reset_index()
    ).rename(columns={"trip_instance_key": f"{bunched_y_n}_trips"})
    return agg1

In [83]:
def agg_final_df(df: pd.DataFrame) -> pd.DataFrame:
    groupby_cols = [
        "caltrans_district",
        "schedule_gtfs_dataset_key",
        "feed_key",
        "organization_name",
        "route_long_name",
        "route_type",
        "route_id",
        "direction_id",
        "stop_id",
        "stop_sequence",
    ]

    # Find total trips that are bunched
    bunched = bunched_not_bunched(df, "bunched", groupby_cols)

    # Find total trips that are NOT bunched
    not_bunched = bunched_not_bunched(df, "not bunched", groupby_cols)

    # Merge
    m1 = pd.merge(not_bunched, bunched, on=groupby_cols, how="outer")

    # Find the % of bunched trips
    m1 = m1.fillna(0)
    m1["all_trips"] = m1.not_bunched_trips + m1.bunched_trips
    m1["per_trip_bunched_per_stop"] = m1.bunched_trips / m1.all_trips

    # Filter out any rows with only one trip of that groupby combo
    # for that service date
    m1 = m1.loc[m1.all_trips > 1].reset_index(drop=True)
    m1 = m1.drop(columns=["not_bunched_trips", "bunched_trips"])

    return m1

In [84]:
transit_matters_m1 = agg_final_df(transit_matters_df1)

In [85]:
# transit_matters_m1 = (
#   transit_matters_m1.sort_values(by=["all_trips"], ascending=False)
#   .drop_duplicates(subset=transit_matters_agg)
#   .reset_index(drop=True)
# )

### Help: Swapped order of a bus is messing with the transit matters metric.
* How to solve for this?? 

In [86]:
preview_cols = [
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
    "pct_actual_schd_headway",
    "bunched_y_n",
]

In [87]:
example2 = transit_matters_df1.loc[
    (transit_matters_df1.stop_id == "2307719")
    & (transit_matters_df1.organization_name == "City of Visalia")
    & (transit_matters_df1.route_id == "2042")
    & (transit_matters_df1.shape_array_key == "60da59c7000ea5dcb5f845d8fa227f14")
]

#### Starting row 33484: the RT Arrival time is swapped. A bus that was scheduled to arrive at 4:27 arrived boefre the bus arrived at 3:42.
* This repeats again row 33486.

In [88]:
example2[preview_cols]

,converted_rt_arrival,actual_arrival_lag_min,converted_schd_arrival,scheduled_arrival_lag_min,pct_actual_schd_headway,bunched_y_n
33471,2024-05-22 06:32:47,NaN,2024-05-22 06:42:00,NaN,NaN,not bunched
33472,2024-05-22 07:26:41,53.90,2024-05-22 07:27:00,45.00,1.20,not bunched
33473,2024-05-22 08:06:11,39.50,2024-05-22 08:12:00,45.00,0.88,not bunched
33474,2024-05-22 08:56:57,50.77,2024-05-22 08:57:00,45.00,1.13,not bunched
33475,2024-05-22 09:37:59,41.03,2024-05-22 09:42:00,45.00,0.91,not bunched
33476,2024-05-22 10:27:26,49.45,2024-05-22 10:27:00,45.00,1.10,not bunched
33477,2024-05-22 11:10:05,42.65,2024-05-22 11:12:00,45.00,0.95,not bunched
33478,2024-05-22 12:01:01,50.93,2024-05-22 11:57:00,45.00,1.13,not bunched
33479,2024-05-22 12:38:08,37.12,2024-05-22 12:42:00,45.00,0.82,not bunched
33480,2024-05-22 13:27:10,49.03,2024-05-22 13:27:00,45.00,1.09,not bunched


### Use 2 minute benchmark
* [Source](https://static1.squarespace.com/static/533b9a24e4b01d79d0ae4376/t/645e82de1f570b31497c44dc/1683915486889/TransitMatters-Headwaymanagement.pdf)
* Justifying the use of
headway maintenance. For example, in April
2022 the 66 bus significantly bunched around
several stops. When bunching is defined as
buses that run within two minutes or less of
each other, inbound buses towards Nubian
Square bunched 10% of the time at Brigham
Circle, 9% at Brookline Village and Roxbury
Crossing, and 8% of the time at Coolidge
Corner. Bunching is even more dramatic
outbound towards Harvard Square where
buses bunched over 35% of the time at Winship
St, 13% at Coolidge Corner and Harvard Ave at
Commonwealth Ave, and 12% at North Harvard
St at Western Ave. View more data about bus
bunching through the TransitMatters Data
Dashboard here.

* To Do: add back in route  & operator information

In [89]:
two_minutes_df = trips_routes_times2.copy()

In [90]:
two_minutes_df["bunched_y_n"] = np.where(
    two_minutes_df["actual_arrival_lag_min"] <= 2, "bunched", "not bunched"
)

In [91]:
two_minutes_df.bunched_y_n.value_counts()

not bunched    33326
bunched          991
Name: bunched_y_n, dtype: int64

In [92]:
final_two_minute = agg_final_df(two_minutes_df)

In [93]:
final_two_minute.loc[
    (final_two_minute.stop_id == "2307695")
    & (final_two_minute.organization_name == "City of Visalia")
    & (final_two_minute.route_id == "2042")
]

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
38,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307695,16,18.00,0.00


### Comparing both outcomes
* There are so many more bunched trips for the 2 minute approach.
* Add back in schedule_gtfs_key and then grab stop level data from the warehouse.

In [94]:
final_two_minute.per_trip_bunched_per_stop.describe(percentiles)

count   675.00
mean      0.02
std       0.02
min       0.00
1%        0.00
2%        0.00
5%        0.00
10%       0.00
50%       0.01
90%       0.05
95%       0.06
98%       0.08
99%       0.09
max       0.11
Name: per_trip_bunched_per_stop, dtype: float64

In [95]:
transit_matters_m1.per_trip_bunched_per_stop.describe(percentiles)

count   675.00
mean      0.03
std       0.05
min       0.00
1%        0.00
2%        0.00
5%        0.00
10%       0.00
50%       0.00
90%       0.10
95%       0.17
98%       0.17
99%       0.17
max       0.19
Name: per_trip_bunched_per_stop, dtype: float64

### Make Visuals

In [96]:
freq_range = [
    "#ccbb44",
    "#e9d868",
    "#fcb40e",
    "#ff9c42",
    "#fc5c04",
    "#dd217d",
    "#dd217d",
    "#dd217d",
]

In [97]:
trips_routes_times2["hour"] = trips_routes_times2["converted_rt_arrival"].dt.hour
trips_routes_times2["min"] = trips_routes_times2["converted_rt_arrival"].dt.minute

In [98]:
trips_routes_times2.head(2)

,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,route_long_name,direction_id,organization_name,caltrans_district,service_date,route_type,feed_key,converted_schd_arrival2,converted_rt_arrival,converted_schd_arrival,delay_min,actual_arrival_lag_min,scheduled_arrival_lag_min,hour,min
0,140959,2,87720.00,0666caf3ec1ecc96b74f4477ee4bc939,7fa809e3c9a51ed0db2af859fc4a43b7,1333,204-13172,013c88496835cd19be42988f8925c274,Metro Local Line,1.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-21 00:22:00,2024-05-22 00:22:13,2024-05-22 00:22:00,0.22,NaN,NaN,0,22
1,140959,2,89520.00,0666caf3ec1ecc96b74f4477ee4bc939,f5140e997b9d121f124da801cfd3c659,3229,204-13172,013c88496835cd19be42988f8925c274,Metro Local Line,1.00,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,2024-05-22,Bus,608992664173210532aa3e6cc573be2f,2024-05-21 00:52:00,2024-05-22 00:53:49,2024-05-22 00:52:00,1.82,31.60,30.00,0,53


In [99]:
def compare_approaches(
    stop_id: str, organization_name: str, route_id: str, stop_sequence: int
):
    transit_matter = transit_matters_m1.loc[
        (transit_matters_m1.stop_id == stop_id)
        & (transit_matters_m1.organization_name == organization_name)
        & (transit_matters_m1.route_id == route_id)
        & (transit_matters_m1.stop_sequence == stop_sequence)
    ]
    display(transit_matter)

    two_min = final_two_minute.loc[
        (final_two_minute.stop_id == stop_id)
        & (final_two_minute.organization_name == organization_name)
        & (final_two_minute.route_id == route_id)
        & (final_two_minute.stop_sequence == stop_sequence)
    ]

    display(two_min)
    total_trips = trips_routes_times2.loc[
        (trips_routes_times2.stop_id == stop_id)
        & (trips_routes_times2.organization_name == organization_name)
        & (trips_routes_times2.route_id == route_id)
        & (trips_routes_times2.stop_sequence == stop_sequence)
    ]

    display(total_trips.trip_instance_key.nunique())

    chart = (
        alt.Chart(total_trips)
        .mark_circle(size=500)
        .encode(
            x="hour",
            y="min",
            color=alt.Color(
                "hour",
                scale=alt.Scale(range=freq_range),
            ),
            tooltip=["hour", "min", "actual_arrival_lag_min"],
        )
        .properties(width=800, height=400)
    )
    display(chart)
    return total_trips

In [100]:
test1 = compare_approaches(
    stop_id="5685",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
    stop_sequence=46,
)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
99,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,204-13172,0.00,5685,46,105.00,0.10


,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
99,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,204-13172,0.00,5685,46,105.00,0.09


105

alt.Chart(...)

In [101]:
test2 = compare_approaches(
    stop_id="2307469",
    organization_name="City of Visalia",
    route_id="2042",
    stop_sequence=27,
)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
28,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307469,27,16.00,0.19


,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
28,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307469,27,16.00,0.00


16

alt.Chart(...)

In [102]:
test3 = compare_approaches(
    stop_id="3104",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=80,
)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
587,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,33-13172,1.00,3104,80,27.00,0.11


,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
587,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,33-13172,1.00,3104,80,27.00,0.04


27

alt.Chart(...)

In [103]:
test4 = compare_approaches(
    stop_id="15320",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=64,
)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
566,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,33-13172,1.00,15320,64,41.00,0.10


,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
566,07 - Los Angeles,0666caf3ec1ecc96b74f4477ee4bc939,608992664173210532aa3e6cc573be2f,Los Angeles County Metropolitan Transportation Authority,Metro Local Line,Bus,33-13172,1.00,15320,64,41.00,0.02


41

alt.Chart(...)

In [104]:
test5 = compare_approaches(
    stop_id="3288014",
    organization_name="City of Thousand Oaks",
    route_id="3402",
    stop_sequence=16,
)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
640,07 - Los Angeles,1770249a5a2e770ca90628434d4934b1,926867fdee73d5fbfe4f011871bcd830,City of Thousand Oaks,Route 11,Bus,3402,0.00,3288014,16,26.00,0.00


,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
640,07 - Los Angeles,1770249a5a2e770ca90628434d4934b1,926867fdee73d5fbfe4f011871bcd830,City of Thousand Oaks,Route 11,Bus,3402,0.00,3288014,16,26.00,0.00


26

alt.Chart(...)

In [105]:
transit_matters_m1.sort_values(by=["per_trip_bunched_per_stop"], ascending=False)

,caltrans_district,schedule_gtfs_dataset_key,feed_key,organization_name,route_long_name,route_type,route_id,direction_id,stop_id,stop_sequence,all_trips,per_trip_bunched_per_stop
28,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307469,27,16.00,0.19
22,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,0.00,2307713,23,17.00,0.18
0,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,0.00,2307414,1,18.00,0.17
26,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307467,25,18.00,0.17
30,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307686,6,18.00,0.17
31,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307687,7,18.00,0.17
32,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307689,10,18.00,0.17
33,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307690,11,18.00,0.17
34,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307691,12,18.00,0.17
35,06 - Fresno,3bda4652977200408690059ef2ec4b4d,0e89d1fd3bd2a09bbbd0d4f79ea5663b,City of Visalia,Route 9,Bus,2042,1.00,2307692,13,18.00,0.17


### Make Maps 
* Think I actually need the vehicle positions since stops are literally the stop's geometry, so it'll always be plotting on the same spot.

* https://github.com/cal-itp/data-analyses/blob/db19b70329f1e817236bda13707dd903c24abb4c/_shared_utils/shared_utils/gtfs_utils_v2.py#L371
* https://github.com/cal-itp/data-analyses/blob/main/gtfs_funnel/download_stops.py

In [106]:
stop

NameError: name 'stop' is not defined

In [ ]:
# What is this file?
vps_gdf = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/vp_2024-05-22.parquet"
)

In [ ]:
vps_gdf = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
        "geometry",
    ]
]

In [ ]:
vps_df = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
    ]
]

In [ ]:
vps_m1 = pd.merge(
    vps_df,
    trips_routes_times2,
    on=["schedule_gtfs_dataset_key", "trip_instance_key"],
    how="inner",
)

In [ ]:
def one_stop(df: pd.DataFrame, stop_id: str, org_name: str, route_id: str):
    # Look at one route & stop
    test_route1 = df.loc[
        (df.organization_name == org_name)
        & (df.route_id == route_id)
        & (df.stop_id == stop_id)
    ]

    compare_approaches(stop_id=stop_id, organization_name=org_name, route_id=route_id)

    # display(test_route1.explore("time_int", marker_kwds = {'radius':20}))
    return test_route1

In [ ]:
metro_test1 = one_stop(
    vps_m1,
    stop_id="5700",
    org_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
)

In [ ]:
metro_test1.shape

In [ ]:
metro_test1.sample(1)

### Other
* https://www.sciencedirect.com/science/article/pii/S1366554523003666
* https://www.sciencedirect.com/science/article/pii/S0968090X22002492?ref=pdf_download&fr=RR-2&rr=8d7d6fb73d8015be